In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [52]:
#Variable for the base url for the LegoShop web
base_url = 'https://www.lego.com'

#Open the LegoShop theme window using Chrome
driver = webdriver.Chrome()
driver.get("https://www.lego.com/en-es/themes")

#Get the content of the openend page and get the HTML
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [132]:
#Indicate every columns name
columns_titles = ["Theme", "Set_Name", "Age", "Pieces", "Rating", "Price"]
#Create the new data frame for the data
df = pd.DataFrame(columns = columns_titles)
df

,Theme,Set_Name,Age,Pieces,Rating,Price


In [134]:
#Get the themes table from the themes page
themes = soup.find("ul", class_="CategoryListingPagestyle__List-sc-880qxz-0 ghsSUv")
theme_title = None
data_frame_index = 0

#Iterate through every element of the found table
for theme in themes.find_all('li'):
    #Get the link for the current element´s page
    link = theme.find('a', class_='Linksstyles__Anchor-sc-83wpk9-0 dfbGrg CategoryLeafstyles__ImagesLink-sc-2bwko3-4 lfBWXz')['href']
    #Get current theme´s name
    title_block = theme.find("span", class_="Markup__StyledMarkup-sc-nc8x20-0 dbPAWk")
    if title_block:
        theme_title = title_block.get_text(strip=True)
    else:
        theme_title = None
    #Get every set from the current theme
    theme_sets_num = getThemeData(link)
    data_frame_index += theme_sets_num

There is only 1 page for this theme(LEGO® Animal Crossing™)
There is only 1 page for this theme(Architecture)
There is only 1 page for this theme(LEGO® Art)
There is only 1 page for this theme(LEGO® Bluey)
There is only 1 page for this theme(LEGO® Braille Bricks)
There is only 1 page for this theme(Despicable Me 4)
There is only 1 page for this theme(LEGO® DUPLO® Peppa Pig)
There is only 1 page for this theme(LEGO® Education)
There is only 1 page for this theme(LEGO® Fortnite)
There is only 1 page for this theme(LEGO® Gabby's Dollhouse)
There is only 1 page for this theme(Jurassic World)
There is only 1 page for this theme(Lord of the Rings™)
There is only 1 page for this theme(Monkie Kid™)
There is only 1 page for this theme(LEGO® ONE PIECE)
There is only 1 page for this theme(Powered UP)
There is only 1 page for this theme(SERIOUS PLAY®)
There is only 1 page for this theme(Sonic the Hedgehog™)
There is only 1 page for this theme(LEGO® The Legend of Zelda™)
There is only 1 page for th

In [128]:
def getSetsData(loaded_page, current_index):
    #Get sets table from the loaded page
    sets = loaded_page.find("ul", id="product-listing-grid")

    #Loop through every element in the table
    for theme_set in sets.find_all("li", class_="Grid_grid-item__Dguxr"):
        current_set_data = [None, None, None, None, None, None]
        #Get set´s title
        current_set_data[0] = theme_title #Set in data_list
        title = theme_set.find("span", class_="markup")
        if title:
            current_set_data[1] = title.get_text(strip=True) #Set in data_list
        #Get stats (Age, Pieces, Rating)
        stats = theme_set.find("div", class_="ProductLeaf_attributesRow__jVFRa")
        if stats:
            stat_index = 2
            for stat in stats.find_all("span"):
                text = stat.get_text(strip=True)
                if not text:
                    continue
                if any(char.isdigit() for char in text):  
                    current_set_data[stat_index] = text #Set in data_list
                stat_index += 1
        #Get the price                    
        price_block = theme_set.find("div", class_ = 'ProductLeaf_priceRow__kwpxi')
        if price_block:
            price_text = price_block.find("span")
            if price_text:
                current_set_data[5] = price_text.get_text(strip=True) #Set in data_list
        if current_set_data[1] is not None and current_set_data[5] is not None:
            df.loc[current_index + data_frame_index] = current_set_data
            current_index += 1

    return current_index


In [93]:
def getThemeData(theme_link):
    #Create the theme page URL
    theme_url = base_url + theme_link

    #Load theme´s main page
    soup = loadPage(theme_url)
    #Get every set from the main page
    set_index = getSetsData(soup, 0)
    
    #Get every theme´s page link
    pages = soup.find('div', class_='Paginationstyles__PageLinks-sc-16f0qal-6 dDoDHg')
    #Check if the theme has more than 1 page
    if pages:    
        for page in pages.find_all('a')[1:]:
            #Get the new page link
            theme_url = base_url + page['href']
            #Load the new page
            soup = loadPage(theme_url)
            #Get every set from the new page
            set_index = getSetsData(soup, set_index)
    else:
        print(f"There is only 1 page for this theme({theme_title})")

    return set_index

In [28]:
def loadPage(link):
    #Open the page using Chrome
    driver = webdriver.Chrome()
    driver.get(link)

    #Wait until the page is open and React has already been loaded
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "product-listing-grid"))
    )

    #Get the HTML from the page
    return BeautifulSoup(driver.page_source, "html.parser")
    

In [137]:
df

,Theme,Set_Name,Age,Pieces,Rating,Price
0,LEGO® Animal Crossing™,Fly with Dodo Airlines,7+,292,4.6,"37,99 €"
1,LEGO® Animal Crossing™,Isabelle's House Visit,6+,389,4.7,"39,99 €"
2,LEGO® Animal Crossing™,Julian's Birthday Party,6+,170,4.9,"14,99 €"
3,LEGO® Animal Crossing™,Bunnie's Outdoor Activities,6+,164,4.9,"19,99 €"
4,LEGO® Animal Crossing™,Stargazing with Celeste,6+,78,4.9,"9,99 €"
5,LEGO® Animal Crossing™,Blathers's Museum Collection,7+,543,4.8,"79,99 €"
6,LEGO® Animal Crossing™,K.K.'s Concert at the Plaza,7+,550,4.8,"79,99 €"
7,LEGO® Animal Crossing™,Able Sisters Clothing Shop,6+,322,4.9,"39,99 €"
8,LEGO® Animal Crossing™,Leif's Caravan & Garden Shop,7+,263,5.0,"29,99 €"
9,LEGO® Animal Crossing™,Goldie's Cosy House,6+,149,4.8,"19,99 €"


In [136]:
pd.set_option('display.max.rows', 1300)

In [139]:
#Export it as a CSV
df.to_csv(r'D:\DataAnalystBootcamp\Lego\LegoSets.csv', index = False)

In [145]:
df

,Theme,Set_Name,Age,Pieces,Rating,Price
0,LEGO® Animal Crossing™,Fly with Dodo Airlines,None,292,4.6,"37,99 €"
1,LEGO® Animal Crossing™,Isabelle's House Visit,None,389,4.7,"39,99 €"
2,LEGO® Animal Crossing™,Julian's Birthday Party,None,170,4.9,"14,99 €"
3,LEGO® Animal Crossing™,Bunnie's Outdoor Activities,None,164,4.9,"19,99 €"
4,LEGO® Animal Crossing™,Stargazing with Celeste,None,78,4.9,"9,99 €"
5,LEGO® Animal Crossing™,Blathers's Museum Collection,None,543,4.8,"79,99 €"
6,LEGO® Animal Crossing™,K.K.'s Concert at the Plaza,None,550,4.8,"79,99 €"
7,LEGO® Animal Crossing™,Able Sisters Clothing Shop,None,322,4.9,"39,99 €"
8,LEGO® Animal Crossing™,Leif's Caravan & Garden Shop,None,263,5.0,"29,99 €"
9,LEGO® Animal Crossing™,Goldie's Cosy House,None,149,4.8,"19,99 €"
